## **Forecasting Persentase Penduduk Miskin Kab Tegal**

Nama    : Feza Raffa Arnanda

NIM     : 222112058

Kelas   : 3SD2

Menggunakan serpapi. Link dokumentasi library : https://serpapi.com/google-trends-interest-over-time

In [1]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=70f74f16e724b144005bdff08babd859f399efe55474ab3b11584ea28002f9ac
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


## Coba Filter

In [ ]:
api_feza = "2bddaed03afb8f3f68c3070a8de76b7a2026179584da15efb63a52103c62bbec"
api_faqih = "cdce8a6f75c63f1bb3835b800e771b7f0785757c7dd50eff3cf7a4317bdd15ee"

In [21]:
from serpapi import GoogleSearch
import pandas as pd

def fetch_data_for_year(start_year, queries):
    params = {
        "engine": "google_trends",
        "q": ", ".join(queries),  # Menggabungkan query menjadi satu string
        "data_type": "GEO_MAP",
        "date": f"{start_year}-01-01 {start_year}-12-31",
        "geo": "ID-JT",
        "region": "CITY",
        "api_key": "cdce8a6f75c63f1bb3835b800e771b7f0785757c7dd50eff3cf7a4317bdd15ee",  # Ganti dengan API key Anda
    }

    search = GoogleSearch(params)
    results_test = search.get_dict()

    # Dictionary untuk menyimpan hasil
    data = {query: [] for query in queries}
    locations = []

    for region in results_test.get('compared_breakdown_by_region', []):
        if region['location'] in ['Slawi', 'Tegal','Adiwerna']:
            locations.append(region['location'])
            for value in region['values']:
                if value['extracted_value'] != '':
                    data[value['query']].append(value['extracted_value'])

    # Membuat DataFrame dari hasil yang didapat
    df = pd.DataFrame({
        'Kota': locations,
    })
    for query in queries:
        df[query] = data[query]

    return df

In [22]:
def save_data_for_years(start_year, end_year, queries, batch_size=5):
    num_batches = (len(queries) + batch_size - 1) // batch_size
    for year in range(start_year, end_year + 1):
        for i in range(num_batches):
            batch_queries = queries[i*batch_size:(i+1)*batch_size]
            df = fetch_data_for_year(year, batch_queries)
            if df.empty:
                print(f"Tidak terdapat data untuk tahun {year} batch {i+1}.")
            else:
                df.to_excel(f"kemiskinan_{year}_batch_{i+1}.xlsx", index=False)
                print(f"Data untuk tahun {year} batch {i+1} berhasil disimpan.")

In [23]:
# Daftar query yang akan digunakan
queries = [
    "Biaya kuliah", "Biaya listrik", "Biaya Pendidikan", "Biaya puskesmas", "Biaya rumah sakit", 
    "Biaya sekolah", "Harga bensin", "Harga LPG", "Harga pampers", "Harga pasta gigi", 
    "Harga sabun mandi", "Harga shampoo", "Harga sikat gigi", "Jual rumah murah", "Jual rumah", 
    "Medical check up", "Paket Internet", "Rumah disewakan", "Zumba"
]

In [24]:
# Menjalankan scraping untuk rentang tahun tertentu
save_data_for_years(start_year=2024, end_year=2024, queries=queries, batch_size=5)

Data untuk tahun 2024 batch 1 berhasil disimpan.
Data untuk tahun 2024 batch 2 berhasil disimpan.
Data untuk tahun 2024 batch 3 berhasil disimpan.
Data untuk tahun 2024 batch 4 berhasil disimpan.


## Compile

In [25]:
import pandas as pd
import os

def compile_files(start_year, end_year, batches):
    compiled_df = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        for batch in batches:
            file_name = f"kemiskinan_{year}_batch_{batch}.xlsx"
            if os.path.exists(file_name):
                df = pd.read_excel(file_name)
                df['Year'] = year  # Tambahkan kolom tahun
                df['Batch'] = batch  # Tambahkan kolom batch
                compiled_df = pd.concat([compiled_df, df], ignore_index=True)
            else:
                print(f"File {file_name} tidak ditemukan.")

    compiled_df.to_excel(f"compiled_data_{start_year}_{end_year}.xlsx", index=False)
    print("Semua file berhasil dikompilasi menjadi compiled_data.xlsx.")

In [26]:
start_year = 2024
end_year = 2024
batches = [1, 2, 3, 4]  # Sesuaikan dengan jumlah batch yang Anda miliki
compile_files(start_year, end_year, batches)

Semua file berhasil dikompilasi menjadi compiled_data.xlsx.


In [20]:
import pandas as pd

# Baca data dari file Excel
file_path = "compiled_data.xlsx"  # Ganti dengan nama file Anda jika berbeda
df = pd.read_excel(file_path)

# Pindahkan kolom "Year" dan "Batch" ke posisi pertama dan kedua
cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Batch')))
cols.insert(0, cols.pop(cols.index('Year')))
df = df[cols]

# Simpan kembali ke file Excel
df.to_excel("compiled_data_reordered.xlsx", index=False)